In [1]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
import copy
from scipy.interpolate import interp1d
import lifelines
from lifelines import CoxPHFitter
dtype = torch.FloatTensor

In [2]:
def sort_data(path):
    data = pd.read_csv(path)
    data.sort_values("OS.time",ascending = False, inplace = True)
    x = data.drop(["Patient_ID", "race_white", "age", "stageh","gradeh", "OS", "OS.time"], axis = 1).values
    ytime = data.loc[:, ["OS.time"]].values
    yevent = data.loc[:, ["OS"]].values
    age = data.loc[:, ["age"]].values
    cstage = data.loc[:, ["stageh"]].values
    hgrade = data.loc[:, ["gradeh"]].values
    race_white = data.loc[:, ["race_white"]].values
    return(x, ytime, yevent, age, cstage, hgrade, race_white)

def load_data(path, dtype):
    x, ytime, yevent, age, cstage, hgrade, race_white = sort_data(path)
    X = torch.from_numpy(x).type(dtype)
    YTIME = torch.from_numpy(ytime).type(dtype)
    YEVENT = torch.from_numpy(yevent).type(dtype)
    AGE = torch.from_numpy(age).type(dtype)
    CSTAGE = torch.from_numpy(cstage).type(dtype)
    HGRADE = torch.from_numpy(hgrade).type(dtype)
    RACE_WHITE = torch.from_numpy(race_white).type(dtype)
    if torch.cuda.is_available():
        X = X.cuda()
        YTIME = YTIME.cuda()
        YEVENT = YEVENT.cuda()
        AGE = AGE.cuda()
        CSTAGE = CSTAGE.cuda()
        HGRADE = HGRADE.cuda()
        RACE_WHITE = RACE_WHITE.cuda()
    return(X, YTIME, YEVENT, AGE, CSTAGE, HGRADE, RACE_WHITE)

In [3]:
x_train, ytime_train, yevent_train, age_train, cstage_train, hgrade_train, race_white_train = load_data("D:/DL/Variational autoencoder/Tryout_01_07_2021/divided_data/exp_1/data_tr_1.csv", dtype)
x_test, ytime_test, yevent_test, age_test, cstage_test, hgrade_test, race_white_test = load_data("D:/DL/Variational autoencoder/Tryout_01_07_2021/divided_data/exp_1/data_tes_1.csv", dtype)

processed_tr_pre = torch.cat((ytime_train, yevent_train, age_train, cstage_train, hgrade_train, race_white_train), 1)
processed_tes_pre = torch.cat((ytime_test, yevent_test, age_test, cstage_test, hgrade_test, race_white_test), 1)

processed_tr = pd.DataFrame(processed_tr_pre, columns = ['OS.time', 'OS.event', 'age', 
                                                         'stageh', 'gradeh', 'race_white'])
processed_tr = processed_tr.astype(float)
processed_tes = pd.DataFrame(processed_tes_pre, columns = ['OS.time', 'OS.event', 'age', 
                                                           'stageh', 'gradeh', 'race_white'])
processed_tes = processed_tes.astype(float)

In [7]:
cph = CoxPHFitter(l1_ratio = 1., penalizer = 0.0001)
cph.fit(processed_tes, duration_col='OS.time', event_col='OS.event')
cph.print_summary()